Извлечение сущностей из документов (новостных текстов)

In [1]:
# Установка всех зависимостей
!pip install -r requirements.txt
!python -m spacy download en_core_web_sm

  Cloning https://github.com/huggingface/accelerate (to revision e1247de01e0733c5d21075cb6f39b2605f4be123) to /private/var/folders/1q/kp4dtr7s5b50s1tnn85mv2bm0000gn/T/pip-install-3acfd_kl/accelerate_aa68812870d84ac986c629f3dbaf2197
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /private/var/folders/1q/kp4dtr7s5b50s1tnn85mv2bm0000gn/T/pip-install-3acfd_kl/accelerate_aa68812870d84ac986c629f3dbaf2197
  Running command git rev-parse -q --verify 'sha^e1247de01e0733c5d21075cb6f39b2605f4be123'
  Running command git fetch -q https://github.com/huggingface/accelerate e1247de01e0733c5d21075cb6f39b2605f4be123
  Running command git checkout -q e1247de01e0733c5d21075cb6f39b2605f4be123
  Resolved https://github.com/huggingface/accelerate to commit e1247de01e0733c5d21075cb6f39b2605f4be123
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import spacy

# Создание примерного файла news_texts.csv
data = {
    "text": [
        "Apple is looking at buying U.K. startup for $1 billion",
        "San Francisco considers banning sidewalk delivery robots",
        "London is a very big city in the United Kingdom",
        "The quick brown fox jumps over the lazy dog",
        "Artificial Intelligence is transforming the tech industry"
    ]
}

df = pd.DataFrame(data)
df.to_csv('news_texts.csv', index=False)

In [2]:
# Пример загрузки данных
data = pd.read_csv('news_texts.csv')
texts = data['text'].tolist()

# Загрузка модели spaCy
nlp = spacy.load('en_core_web_sm')


# Функция для извлечения сущностей
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities


# Применение функции ко всем текстам
data['entities'] = data['text'].apply(extract_entities)

# Сохранение результатов
data.to_csv('entities_extracted.csv', index=False)


Теория для VLM
Визуальные языковые модели (VLM) представляют собой специализированные системы искусственного интеллекта, которые учатся понимать визуальные данные, такие как изображения и графики, в контексте текстовых запросов или описаний. Эти модели основаны на глубоком обучении и обычно используют архитектуры, подобные тем, что успешно применяются для обработки текста, например, Transformer 

Процесс обучения VLM начинается с предварительного обучения на больших наборах данных, где модель учится распознавать паттерны и взаимосвязи между визуальными и текстовыми данными. Это включает анализ изображений и соответствующих текстовых описаний или ответов на вопросы о визуальных данных

Выбор модели VLM
Для задачи Chart Question Answering рассмотрим следующие модели: 
* LXMERT 
* VisualBERT 
* ViLT (Vision-and-Language Transformer)

ViLT отличается простотой в дообучении и инференсе, а также возможностью квантизации
Архитектура VLM: 
1. Визуальная часть: CNN, ResNet, EfficientNet, ViT. 
2. Текстовая часть: Трансформеры (BERT, GPT). 
3. Мультимодальная часть: Интеграция визуальных и текстовых данных с помощью attention mechanisms (LXMERT, VisualBERT, ViLBERT)

Обучение VLM: 
Предварительное обучение: Обучение на больших объемах данных. Тонкая настройка: Дообучение на специфических датасетах. 
Дополнительные методы: Transfer learning, квантизация, оптимизация

Выбор датасета 
Для задачи Chart Question Answering: ChartMimic: Специально созданный для анализа графиков и диаграмм ChartMimic на Huggingface ChartLlama-Dataset: Для задач вопрос-ответ по графикам ChartLlama-Dataset на Huggingface

In [41]:
from datasets.formatting.formatting import LazyBatch
import requests
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd

# Determine device
# Raise RuntimeError: slow_conv2d_forward_mps: input(device='cpu') and weight(device=mps:0')  must be on the same device
device = torch.device("cpu")

# Load model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load dataset, data load_dataset not working and i use get request
response = requests.get(
    url="https://datasets-server.huggingface.co/first-rows?dataset=listen2you002%2FChartLlama-Dataset&config=default&split=train").json()

dataset = Dataset.from_pandas(pd.DataFrame(row['row'] for row in response['rows']))


# Function to load image using PIL
def load_image(image_path):
    """
    Loads an image and returns it as a numpy array.
    Args:
        image_path: 

    Returns:

    """
    return Image.open(image_path)


# Data preprocessing

def preprocess(examples: LazyBatch):
    """
    Preprocessing step of the dataset.
    Args:
        examples: 

    Returns:
        
    """
    images = [load_image(image) for image in examples['image']]
    text = [",".join(asc["value"] for asc in row) for row in examples['conversations']]
    inputs = processor(text=text, images=images, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}
    return inputs


# Apply preprocessing to the dataset
dataset = dataset.map(preprocess, batched=True)

# Split into train and test datasets
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    output_dir='./clip-chartmimic'
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

['image', 'conversations', 'id', 'model']
2
3


RuntimeError: slow_conv2d_forward_mps: input(device='cpu') and weight(device=mps:0')  must be on the same device

Выбор метрики: Accuracy (точность)

In [6]:
# Оценка точности модели до и после дообучения
eval_results = trainer.evaluate()
print(f"Точность модели после дообучения: {eval_results['eval_accuracy']}")


RuntimeError: slow_conv2d_forward_mps: input(device='cpu') and weight(device=mps:0')  must be on the same device